# Insurance Customer Segmentation and Fraud Detection

## Data Scientist: Nikita Ramsey

### Project overview:

### About the data:



*  Month: The month in which the insurance claim was made.
*  WeekOfMonth: The week of the month in which the insurance claim was made.
*  DayOfWeek: The day of the week on which the insurance claim was made.
*  Make: The manufacturer of the vehicle involved in the claim.
*  AccidentArea: The area where the accident occurred (e.g., urban, rural).
*  DayOfWeekClaimed: The day of the week on which the insurance claim was processed.
*  MonthClaimed: The month in which the insurance claim was processed.
*  WeekOfMonthClaimed: The week of the month in which the insurance claim was processed.
*  Sex: The gender of the policyholder.
*  MaritalStatus: The material status of the policyholder.
*  Age: The age of the policyholder.
*  Fault: Indicates whether the policyholder was at fault in the accident.
*  PolicyType: The type of insurance policy (e.g., comprehensive, third-party).
*  VehicleCategory: The category of the vehicle (e.g., sedan, SUV).
*  VehiclePrice: The price of vehicle.
*  FraudFound_P: Indicates whether fraud was detected in the insurance claim.
*  PolicyNumber: The unique identifier for the insurance policy.
*  RepNumber: The unique identifier for the insurance representative handling the claim.
*  Deductible: The amount that the policy holder must pay out of pocket before the insurance company pays the remaining costs.
*  DriverRating: The rating of the driver, often based on driving history or other factors.
*  Days_Policy_Accident: The number of days since the policy was issued until the accident occurred.
*  Days_Policy_Claim: The number of days since the policy was issued until the claim was made.
*  PastNumberOfClaims: The number of claims previously made by the policyholder.
*  AgeOfVehicle: The age of the vehicle involved in the claim.
*  AgeOfPolicyHolder: The age of the policyholder.
*  PoliceReportFiled: Indicates whether a police report was filed for the accident.
*  WitnessPresent: Indicates whether a witness was present at the scene of the accident.
*  AgentType: The type of insurance agent handling the policy (e.g., internal, external)
*  NumberOfSuppliments: The number of supplementary documents or claims related to the main claim, categorized into ranges.
*  AddressChange_Claim: Indicates whether the address of the policyholder was changed at the time of the claim, categorized into ranges.
*  NumberOfCars: The number of cars insured under the policy, categorized into ranges.
*  Year: The year in which the claim was made or processed.
*  BasePolicy: The base policy type (e.g., Liability, Collision, All Perils).




### Load libraries:

In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')

import shutil
from pathlib import Path
import scipy.stats as stats
from scipy import sparse
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score, StratifiedKFold, cross_validate, learning_curve
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, confusion_matrix, RocCurveDisplay, roc_auc_score, roc_curve, recall_score, accuracy_score, precision_score, f1_score  
from sklearn.pipeline import Pipeline, FunctionTransformer, FeatureUnion
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance

### Load and sanity-check the data:

In [42]:
# load data
pd.set_option('display.max_columns', None)
df = pd.read_csv('data\\fraud_oracle.csv')
df.head()

,Month,WeekOfMonth,DayOfWeek,Make,AccidentArea,DayOfWeekClaimed,MonthClaimed,WeekOfMonthClaimed,Sex,MaritalStatus,Age,Fault,PolicyType,VehicleCategory,VehiclePrice,FraudFound_P,PolicyNumber,RepNumber,Deductible,DriverRating,Days_Policy_Accident,Days_Policy_Claim,PastNumberOfClaims,AgeOfVehicle,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange_Claim,NumberOfCars,Year,BasePolicy
0,Dec,5,Wednesday,Honda,Urban,Tuesday,Jan,1,Female,Single,21,Policy Holder,Sport - Liability,Sport,more than 69000,0,1,12,300,1,more than 30,more than 30,none,3 years,26 to 30,No,No,External,none,1 year,3 to 4,1994,Liability
1,Jan,3,Wednesday,Honda,Urban,Monday,Jan,4,Male,Single,34,Policy Holder,Sport - Collision,Sport,more than 69000,0,2,15,400,4,more than 30,more than 30,none,6 years,31 to 35,Yes,No,External,none,no change,1 vehicle,1994,Collision
2,Oct,5,Friday,Honda,Urban,Thursday,Nov,2,Male,Married,47,Policy Holder,Sport - Collision,Sport,more than 69000,0,3,7,400,3,more than 30,more than 30,1,7 years,41 to 50,No,No,External,none,no change,1 vehicle,1994,Collision
3,Jun,2,Saturday,Toyota,Rural,Friday,Jul,1,Male,Married,65,Third Party,Sedan - Liability,Sport,20000 to 29000,0,4,4,400,2,more than 30,more than 30,1,more than 7,51 to 65,Yes,No,External,more than 5,no change,1 vehicle,1994,Liability
4,Jan,5,Monday,Honda,Urban,Tuesday,Feb,2,Female,Single,27,Third Party,Sport - Collision,Sport,more than 69000,0,5,3,400,1,more than 30,more than 30,none,5 years,31 to 35,No,No,External,none,no change,1 vehicle,1994,Collision


In [43]:
# sample a few rows
df.sample(10)

,Month,WeekOfMonth,DayOfWeek,Make,AccidentArea,DayOfWeekClaimed,MonthClaimed,WeekOfMonthClaimed,Sex,MaritalStatus,Age,Fault,PolicyType,VehicleCategory,VehiclePrice,FraudFound_P,PolicyNumber,RepNumber,Deductible,DriverRating,Days_Policy_Accident,Days_Policy_Claim,PastNumberOfClaims,AgeOfVehicle,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange_Claim,NumberOfCars,Year,BasePolicy
13055,Apr,4,Friday,Honda,Urban,Tuesday,May,1,Female,Single,31,Third Party,Sedan - Liability,Sport,20000 to 29000,0,13056,2,400,2,8 to 15,more than 30,2 to 4,5 years,31 to 35,No,No,External,none,no change,1 vehicle,1996,Liability
3088,Apr,5,Friday,Toyota,Urban,Wednesday,May,2,Male,Married,28,Policy Holder,Sedan - All Perils,Sedan,more than 69000,0,3089,1,400,2,more than 30,more than 30,1,7 years,31 to 35,No,No,External,more than 5,no change,1 vehicle,1994,All Perils
1230,Jan,4,Tuesday,Toyota,Urban,Friday,Apr,2,Female,Single,34,Policy Holder,Sedan - Liability,Sport,20000 to 29000,0,1231,10,400,3,more than 30,more than 30,2 to 4,5 years,31 to 35,No,No,External,none,no change,3 to 4,1994,Liability
12647,Sep,4,Sunday,Pontiac,Urban,Monday,Sep,4,Male,Married,30,Policy Holder,Sedan - Collision,Sedan,20000 to 29000,0,12648,11,400,3,more than 30,more than 30,more than 4,5 years,31 to 35,No,No,External,more than 5,no change,1 vehicle,1996,Collision
7685,Dec,2,Saturday,Mazda,Rural,Monday,Dec,2,Female,Divorced,38,Policy Holder,Sedan - Collision,Sedan,20000 to 29000,0,7686,13,400,4,more than 30,more than 30,1,7 years,36 to 40,No,No,External,none,no change,1 vehicle,1995,Collision
6215,Dec,1,Thursday,Chevrolet,Urban,Wednesday,Dec,2,Male,Married,26,Policy Holder,Sedan - Liability,Sport,30000 to 39000,0,6216,13,400,1,more than 30,more than 30,none,5 years,31 to 35,No,No,External,none,no change,1 vehicle,1995,Liability
9186,Oct,3,Tuesday,Mazda,Urban,Friday,Oct,4,Female,Married,35,Third Party,Sedan - Collision,Sedan,30000 to 39000,0,9187,12,400,2,more than 30,more than 30,none,7 years,31 to 35,No,No,External,none,no change,1 vehicle,1995,Collision
9546,Jun,4,Sunday,Honda,Urban,Tuesday,Jun,4,Male,Married,35,Policy Holder,Sedan - Liability,Sport,20000 to 29000,0,9547,13,400,1,more than 30,more than 30,2 to 4,6 years,31 to 35,No,No,External,none,no change,1 vehicle,1995,Liability
3136,Sep,4,Wednesday,Chevrolet,Urban,Friday,Sep,4,Male,Married,52,Policy Holder,Sedan - All Perils,Sedan,20000 to 29000,0,3137,3,400,2,more than 30,more than 30,1,more than 7,41 to 50,No,No,External,none,no change,1 vehicle,1994,All Perils
9246,Jan,2,Wednesday,Chevrolet,Urban,Monday,Jan,3,Male,Married,32,Policy Holder,Sedan - All Perils,Sedan,20000 to 29000,1,9247,2,400,3,more than 30,more than 30,none,6 years,31 to 35,No,No,External,none,no change,1 vehicle,1995,All Perils


In [44]:
# review the data structure
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15420 entries, 0 to 15419
Data columns (total 33 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Month                 15420 non-null  object
 1   WeekOfMonth           15420 non-null  int64 
 2   DayOfWeek             15420 non-null  object
 3   Make                  15420 non-null  object
 4   AccidentArea          15420 non-null  object
 5   DayOfWeekClaimed      15420 non-null  object
 6   MonthClaimed          15420 non-null  object
 7   WeekOfMonthClaimed    15420 non-null  int64 
 8   Sex                   15420 non-null  object
 9   MaritalStatus         15420 non-null  object
 10  Age                   15420 non-null  int64 
 11  Fault                 15420 non-null  object
 12  PolicyType            15420 non-null  object
 13  VehicleCategory       15420 non-null  object
 14  VehiclePrice          15420 non-null  object
 15  FraudFound_P          15420 non-null

In [45]:
# get names of columns
df.columns.tolist()

['Month',
 'WeekOfMonth',
 'DayOfWeek',
 'Make',
 'AccidentArea',
 'DayOfWeekClaimed',
 'MonthClaimed',
 'WeekOfMonthClaimed',
 'Sex',
 'MaritalStatus',
 'Age',
 'Fault',
 'PolicyType',
 'VehicleCategory',
 'VehiclePrice',
 'FraudFound_P',
 'PolicyNumber',
 'RepNumber',
 'Deductible',
 'DriverRating',
 'Days_Policy_Accident',
 'Days_Policy_Claim',
 'PastNumberOfClaims',
 'AgeOfVehicle',
 'AgeOfPolicyHolder',
 'PoliceReportFiled',
 'WitnessPresent',
 'AgentType',
 'NumberOfSuppliments',
 'AddressChange_Claim',
 'NumberOfCars',
 'Year',
 'BasePolicy']

Descriptive (numerical) considerations and insights:

*  Data doesn't appear to have any extreme values, so outlier removal techniques won't be necessary
*  Claims are often made and processed during the same week
*  Average driver age is ~ 39.9 years, which includes those under the age of 18
*  Average deductible is ~ $407.70
*  Average driver rating is ~ 2.5, on a 1-4 scale
*  Most claims were made or processed in 1994
*  Fraud only occurs in a very, very small number of claims, as one would expect

In [46]:
# descriptive stats for numerical features
df.describe().T.round(3)

,count,mean,std,min,25%,50%,75%,max
WeekOfMonth,15420.0,2.789,1.288,1.0,2.00,3.0,4.00,5.0
WeekOfMonthClaimed,15420.0,2.694,1.259,1.0,2.00,3.0,4.00,5.0
Age,15420.0,39.856,13.492,0.0,31.00,38.0,48.00,80.0
FraudFound_P,15420.0,0.060,0.237,0.0,0.00,0.0,0.00,1.0
PolicyNumber,15420.0,7710.500,4451.515,1.0,3855.75,7710.5,11565.25,15420.0
RepNumber,15420.0,8.483,4.600,1.0,5.00,8.0,12.00,16.0
Deductible,15420.0,407.704,43.951,300.0,400.00,400.0,400.00,700.0
DriverRating,15420.0,2.488,1.119,1.0,1.00,2.0,3.00,4.0
Year,15420.0,1994.866,0.803,1994.0,1994.00,1995.0,1996.00,1996.0


Descriptive (categorical) considerations and insights:

*  Will need to review DayOfWeekClaimed and MonthClaimed, as the number of unique values exceed expectation (based on number of calendar days and months)
*  When looking at categories with large (instead of marginal) majorities, most claims:
    *  occur on Monday
    *  occur in urban areas
    *  involve married men who are at fault and have multiple previous claims
    *  involve lower priced sedans that are 7 years old, with collision policies
    *  did not have a witness present or have a police report filed
    *  occur at least a month after the policy was issued

In [47]:
# descriptive stats for categorical features
df.describe(include="object").T

,count,unique,top,freq
Month,15420,12,Jan,1411
DayOfWeek,15420,7,Monday,2616
Make,15420,19,Pontiac,3837
AccidentArea,15420,2,Urban,13822
DayOfWeekClaimed,15420,8,Monday,3757
MonthClaimed,15420,13,Jan,1446
Sex,15420,2,Male,13000
MaritalStatus,15420,4,Married,10625
Fault,15420,2,Policy Holder,11230
PolicyType,15420,9,Sedan - Collision,5584


In reviewing the DayOfWeekClaimed and MonthClaimed features, '0' values were included which will be removed. All other features appear ok, but will need to have some type conversions.

In [48]:
# list the different categories, to check for typos, casing and unexpected values
for col in df.select_dtypes(include='object'):
    print(f'{col}: {sorted(df[col].unique())}\n')

Month: ['Apr', 'Aug', 'Dec', 'Feb', 'Jan', 'Jul', 'Jun', 'Mar', 'May', 'Nov', 'Oct', 'Sep']

DayOfWeek: ['Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday']

Make: ['Accura', 'BMW', 'Chevrolet', 'Dodge', 'Ferrari', 'Ford', 'Honda', 'Jaguar', 'Lexus', 'Mazda', 'Mecedes', 'Mercury', 'Nisson', 'Pontiac', 'Porche', 'Saab', 'Saturn', 'Toyota', 'VW']

AccidentArea: ['Rural', 'Urban']

DayOfWeekClaimed: ['0', 'Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday']

MonthClaimed: ['0', 'Apr', 'Aug', 'Dec', 'Feb', 'Jan', 'Jul', 'Jun', 'Mar', 'May', 'Nov', 'Oct', 'Sep']

Sex: ['Female', 'Male']

MaritalStatus: ['Divorced', 'Married', 'Single', 'Widow']

Fault: ['Policy Holder', 'Third Party']

PolicyType: ['Sedan - All Perils', 'Sedan - Collision', 'Sedan - Liability', 'Sport - All Perils', 'Sport - Collision', 'Sport - Liability', 'Utility - All Perils', 'Utility - Collision', 'Utility - Liability']

VehicleCategory: ['Sedan', 'Sport', 'Utility']


Data doesn't have any missing values.

In [49]:
# check for na values
df.isna().sum().sort_values(ascending=False)

Month                   0
WeekOfMonth             0
DayOfWeek               0
Make                    0
AccidentArea            0
DayOfWeekClaimed        0
MonthClaimed            0
WeekOfMonthClaimed      0
Sex                     0
MaritalStatus           0
Age                     0
Fault                   0
PolicyType              0
VehicleCategory         0
VehiclePrice            0
FraudFound_P            0
PolicyNumber            0
RepNumber               0
Deductible              0
DriverRating            0
Days_Policy_Accident    0
Days_Policy_Claim       0
PastNumberOfClaims      0
AgeOfVehicle            0
AgeOfPolicyHolder       0
PoliceReportFiled       0
WitnessPresent          0
AgentType               0
NumberOfSuppliments     0
AddressChange_Claim     0
NumberOfCars            0
Year                    0
BasePolicy              0
dtype: int64

Data doesn't have any duplicate rows.

In [50]:
# check for dups
df.duplicated().sum()

np.int64(0)

### Data cleaning:

A few different updates need to be made before starting EDA, including:

*  dropping unique identifiers
*  dropping '0' values from the DayOfWeekClaimed and MonthClaimed features
*  type conversions 

In [51]:
# drop unique identifiers ()
df = df.drop(columns=['RepNumber', 'PolicyNumber'])
df.head()

,Month,WeekOfMonth,DayOfWeek,Make,AccidentArea,DayOfWeekClaimed,MonthClaimed,WeekOfMonthClaimed,Sex,MaritalStatus,Age,Fault,PolicyType,VehicleCategory,VehiclePrice,FraudFound_P,Deductible,DriverRating,Days_Policy_Accident,Days_Policy_Claim,PastNumberOfClaims,AgeOfVehicle,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange_Claim,NumberOfCars,Year,BasePolicy
0,Dec,5,Wednesday,Honda,Urban,Tuesday,Jan,1,Female,Single,21,Policy Holder,Sport - Liability,Sport,more than 69000,0,300,1,more than 30,more than 30,none,3 years,26 to 30,No,No,External,none,1 year,3 to 4,1994,Liability
1,Jan,3,Wednesday,Honda,Urban,Monday,Jan,4,Male,Single,34,Policy Holder,Sport - Collision,Sport,more than 69000,0,400,4,more than 30,more than 30,none,6 years,31 to 35,Yes,No,External,none,no change,1 vehicle,1994,Collision
2,Oct,5,Friday,Honda,Urban,Thursday,Nov,2,Male,Married,47,Policy Holder,Sport - Collision,Sport,more than 69000,0,400,3,more than 30,more than 30,1,7 years,41 to 50,No,No,External,none,no change,1 vehicle,1994,Collision
3,Jun,2,Saturday,Toyota,Rural,Friday,Jul,1,Male,Married,65,Third Party,Sedan - Liability,Sport,20000 to 29000,0,400,2,more than 30,more than 30,1,more than 7,51 to 65,Yes,No,External,more than 5,no change,1 vehicle,1994,Liability
4,Jan,5,Monday,Honda,Urban,Tuesday,Feb,2,Female,Single,27,Third Party,Sport - Collision,Sport,more than 69000,0,400,1,more than 30,more than 30,none,5 years,31 to 35,No,No,External,none,no change,1 vehicle,1994,Collision


In [52]:
# drop '0' values from DayOfWeekClaimed and MonthClaimed features
df = df[(df['DayOfWeekClaimed'] != '0') & (df['MonthClaimed'] != '0')]  
zero = df[(df['DayOfWeekClaimed'] == '0') | (df['MonthClaimed'] == '0')]    
print(zero.shape)

(0, 31)


In [53]:
# seperate categorical features
ordinal_cols = [
    "VehiclePrice",
    "Days_Policy_Accident",
    "Days_Policy_Claim",
    "PastNumberOfClaims",
    "AgeOfVehicle",
    "AgeOfPolicyHolder",
    "NumberOfSuppliments",
    "AddressChange_Claim",
    "NumberOfCars",
    "DriverRating",
    "Deductible",
    "Year",
    "WeekOfMonth",
    "WeekOfMonthClaimed"
]

nominal_cols = [
    "Make",
    "AccidentArea",
    "Sex",
    "MaritalStatus",
    "Fault",
    "PolicyType",
    "VehicleCategory",
    "PoliceReportFiled",
    "WitnessPresent",
    "AgentType",
    "BasePolicy"
]

cyclic_cols = [
    "Month",
    "MonthClaimed",
    "DayOfWeek",
    "DayOfWeekClaimed"
]

In [54]:
# convert nominal data to category dtype
for col in nominal_cols:
    df[col] = df[col].astype('category')
df[nominal_cols].dtypes

Make                 category
AccidentArea         category
Sex                  category
MaritalStatus        category
Fault                category
PolicyType           category
VehicleCategory      category
PoliceReportFiled    category
WitnessPresent       category
AgentType            category
BasePolicy           category
dtype: object

In [55]:
# convert ordinal data to category dtype with order
for col in ordinal_cols:
    df[col] = df[col].astype('category').cat.as_ordered()
df[ordinal_cols].dtypes

VehiclePrice            category
Days_Policy_Accident    category
Days_Policy_Claim       category
PastNumberOfClaims      category
AgeOfVehicle            category
AgeOfPolicyHolder       category
NumberOfSuppliments     category
AddressChange_Claim     category
NumberOfCars            category
DriverRating            category
Deductible              category
Year                    category
WeekOfMonth             category
WeekOfMonthClaimed      category
dtype: object

In [56]:
# convert target to category dtype
df['FraudFound_P'] = df['FraudFound_P'].astype('category')
df['FraudFound_P'].dtypes

CategoricalDtype(categories=[0, 1], ordered=False, categories_dtype=int64)

In [57]:
# set proper ordering for month and day of week for type conversion
month_order = ["Jan","Feb","Mar","Apr","May","Jun", "Jul","Aug","Sep","Oct","Nov","Dec"]

dow_order = ["Monday","Tuesday","Wednesday","Thursday", "Friday","Saturday","Sunday"]

In [58]:
# apply ordering
df["Month"] = pd.Categorical(df["Month"], categories=month_order, ordered=True)
df["MonthClaimed"] = pd.Categorical(df["MonthClaimed"], categories=month_order, ordered=True)
df["DayOfWeek"] = pd.Categorical(df["DayOfWeek"], categories=dow_order, ordered=True)
df["DayOfWeekClaimed"] = pd.Categorical(df["DayOfWeekClaimed"], categories=dow_order, ordered=True)

Now that the data has been cleaned, we'll do a quick review of proportions to identify rare types prior to EDA.

In [59]:
# calculate category proportions to identify most common and most rare categories
for c in nominal_cols:
    display((df[c].value_counts(normalize=True) * 100).round(3).head(10))

Make
Pontiac      24.885
Toyota       20.241
Honda        18.159
Mazda        15.267
Chevrolet    10.902
Accura        3.061
Ford          2.918
VW            1.835
Dodge         0.707
Saab          0.700
Name: proportion, dtype: float64

AccidentArea
Urban    89.643
Rural    10.357
Name: proportion, dtype: float64

Sex
Male      84.305
Female    15.695
Name: proportion, dtype: float64

MaritalStatus
Married     68.908
Single      30.372
Divorced     0.493
Widow        0.227
Name: proportion, dtype: float64

Fault
Policy Holder    72.826
Third Party      27.174
Name: proportion, dtype: float64

...

VehicleCategory
Sedan      62.715
Sport      34.749
Utility     2.536
Name: proportion, dtype: float64

PoliceReportFiled
No     97.224
Yes     2.776
Name: proportion, dtype: float64

WitnessPresent
No     99.436
Yes     0.564
Name: proportion, dtype: float64

AgentType
External    98.437
Internal     1.563
Name: proportion, dtype: float64

BasePolicy
Collision     38.667
Liability     32.486
All Perils    28.848
Name: proportion, dtype: float64

In [60]:
# calculate category proportions to identify most common and most rare categories
for c in ordinal_cols:
    display((df[c].value_counts(normalize=True) * 100).round(3).head(10))

VehiclePrice
20000 to 29000     52.396
30000 to 39000     22.913
more than 69000    14.028
less than 20000     7.108
40000 to 59000      2.990
60000 to 69000      0.564
Name: proportion, dtype: float64

Days_Policy_Accident
more than 30    98.878
none             0.357
8 to 15          0.357
15 to 30         0.318
1 to 7           0.091
Name: proportion, dtype: float64

Days_Policy_Claim
more than 30    99.501
15 to 30         0.363
8 to 15          0.136
Name: proportion, dtype: float64

PastNumberOfClaims
2 to 4         35.573
none           28.218
1              23.173
more than 4    13.036
Name: proportion, dtype: float64

AgeOfVehicle
7 years        37.661
more than 7    25.819
6 years        22.362
5 years         8.801
new             2.413
4 years         1.485
3 years         0.986
2 years         0.473
Name: proportion, dtype: float64

...

DriverRating
1    25.579
3    25.190
2    24.645
4    24.587
Name: proportion, dtype: float64

Deductible
400    96.225
700     2.017
500     1.706
300     0.052
Name: proportion, dtype: float64

Year
1994    39.827
1995    33.692
1996    26.480
Name: proportion, dtype: float64

WeekOfMonth
3    23.607
2    23.069
4    22.038
1    20.669
5    10.617
Name: proportion, dtype: float64

WeekOfMonthClaimed
2    24.126
3    23.238
1    22.369
4    22.265
5     8.003
Name: proportion, dtype: float64

In [61]:
# calculate category proportions to identify most common and most rare categories
for c in cyclic_cols:
    display((df[c].value_counts(normalize=True) * 100).round(3).head(10))

Month
Jan    9.151
May    8.866
Mar    8.820
Jun    8.567
Oct    8.464
Dec    8.334
Apr    8.301
Feb    8.211
Jul    8.146
Sep    8.042
Name: proportion, dtype: float64

MonthClaimed
Jan    9.378
May    9.151
Mar    8.742
Oct    8.684
Jun    8.386
Feb    8.347
Nov    8.334
Apr    8.243
Sep    8.055
Jul    7.945
Name: proportion, dtype: float64

DayOfWeek
Monday       16.960
Friday       15.857
Tuesday      14.917
Thursday     14.093
Wednesday    14.002
Saturday     12.854
Sunday       11.317
Name: proportion, dtype: float64

DayOfWeekClaimed
Monday       24.366
Tuesday      21.889
Wednesday    19.139
Thursday     17.251
Friday       16.194
Saturday      0.824
Sunday        0.337
Name: proportion, dtype: float64

Rare types insights:

*  Identifying rare types can be informative in fraud detection tasks.
*  For the nominal data, some of the rarest types include:
    *  Make: VW, Dodge, Saab
    *  MartialStatus: Divorced, Single
    *  PolicyType: Utility - Collision, Sport - All Perils, Utility - Liability, Sport - Liability
    *  VehicleCategory: Utility
    *  PoliceReportFiled: No
    *  WitnessPresent: No
    *  AgentType: Internal
*  For the ordinal data, some of the rarest types include:
    *  VehiclePrice: 40000 to 59000, 60000 to 69000
    *  Days_Policy_Accident: none, 8 to 15, 15 to 30, 1 to 7
    *  Days_Policy_Claimed: 15 to 30, 8 to 15
    *  AgeOfVehicle: 3 years, 2 years
    *  AgeOfPolicyHolder: 21 to 25, 18 to 20
    *  AddressChange_Claim: 2 to 3 years, 1 year, under 6 months
    *  NumberOfCars: 3 to 4, 5 to 8, more than 8
    *  Deductible: 700, 500, 300
*  Most of the cyclic features are relatively uniform, with the exception of DayOfWeekClaimed, which appears right-skewed

## Exploratory Data Analysis:

*  Univariate Analysis

    1. Categorical features: Barplots
    2. Numerical features: Histograms (w/ KDE) and Boxplots/Violinplots

*  Bivariate Analysis

    1. Categorical vs Target: Heatmap of crosstabs
    2. Numerical vs Target: Boxplots/Violinplots or Swarm plots
    3. Numeric vs Numeric: Hexbin plots (for selected features)
    4. Behavioral relationships with Target: binned churn-rate curves
    5. Effect size: mean diffs/ SMD (standardized mean diffs)

*  Correlation Analysis

### Univariate Analysis

### Bivariate Analysis

In [ ]:
# create mappings for month and day of week for type conversion
month_map = {
    "Jan": 1, "Feb": 2, "Mar": 3, "Apr": 4,
    "May": 5, "Jun": 6, "Jul": 7, "Aug": 8,
    "Sep": 9, "Oct": 10, "Nov": 11, "Dec": 12
}

dow_map = {
    "Monday": 1, "Tuesday": 2, "Wednesday": 3, "Thursday": 4,
    "Friday": 5, "Saturday": 6, "Sunday": 7
}

In [ ]:
# convert cyclic features to numerical dtype using the mappings
for col in cyclic_cols:
    if 'Month' in col:
        df[col] = df[col].map(month_map).astype(int)
    elif 'DayOfWeek' in col:
        df[col] = df[col].map(dow_map).astype(int)

### Correlation Analysis

### Exploratory Data Analysis Summary

## Experimentatl Design: